# 文件合并

In [1]:
import pandas as pd

# 定义要合并的 Excel 文件列表
excel_files = [
    r"C:\Users\wangz\Downloads\processed\part_1_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_2_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_3_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_4_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_5_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_6_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_7_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_8_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_9_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_10_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_11_translated.xlsx",
]

# 读取并合并所有 Excel 文件
df_list = [pd.read_excel(file) for file in excel_files]
merged_df = pd.concat(df_list, ignore_index=True)

# 保存合并后的 DataFrame 到新的 Excel 文件
merged_df.to_excel(r"D:\Projects\ai-translator\src\multilangInitData20250210空_translated.xlsx", index=False)


FileCreateError: [WinError 2] 系统找不到指定的文件。: 'C:\\Users\\wangz\\AppData\\Local\\Temp\\tmp2dbjpq31'

# 文件拆分

In [1]:
import os
import pandas as pd

def split_excel(input_file, output_folder, chunk_size=100000):
    """
    将一个Excel文件按指定行数拆分为多个Excel文件，并存储到指定文件夹。

    Args:
        input_file (str): 输入的Excel文件路径。
        output_folder (str): 输出文件夹路径。
        chunk_size (int): 每个拆分Excel文件包含的最大行数，默认10万。
    """
    # 检查输出文件夹是否存在，不存在则创建
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 读取Excel文件（假设默认是Sheet1）
    print("正在读取Excel文件...")
    excel_data = pd.read_excel(input_file, sheet_name=0)

    # 获取总行数
    total_rows = excel_data.shape[0]
    print(f"Excel文件总行数: {total_rows} 行")

    # 按chunk_size进行分割
    for i in range(0, total_rows, chunk_size):
        chunk = excel_data.iloc[i:i + chunk_size]  # 提取数据块
        output_file = os.path.join(output_folder, f"part_{i // chunk_size + 1}.xlsx")
        chunk.to_excel(output_file, index=False, engine='openpyxl')  # 保存为Excel
        print(f"已保存: {output_file}")
    
    print("拆分完成！")

# 示例用法
if __name__ == "__main__":
    input_file = r"C:\Users\wangz\Downloads\开放平台内容多语.xlsx"  # 输入文件路径
    output_folder = r"D:\Projects\ai-translator\src\开放平台内容多语"  # 输出文件夹路径
    split_excel(input_file, output_folder)


正在读取Excel文件...
Excel文件总行数: 302099 行
已保存: D:\Projects\ai-translator\src\开放平台内容多语\part_1.xlsx
已保存: D:\Projects\ai-translator\src\开放平台内容多语\part_2.xlsx
已保存: D:\Projects\ai-translator\src\开放平台内容多语\part_3.xlsx
已保存: D:\Projects\ai-translator\src\开放平台内容多语\part_4.xlsx
拆分完成！


# 友互通翻译

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import zhconv

load_dotenv()
client = OpenAI()
target_columns = [
    "繁體中文",
    "印尼语",
    "匈牙利语",
    "葡萄牙语",
    "泰语",
    "土耳其语",
    "越南语",
    "俄语",
    "阿拉伯语",
    "芬兰语",
    "丹麦语",
    "荷兰语",
    "波兰语",
    "法语",
    "德语",
    "日语",
    "挪威语",
    "希伯来语",
    "韩语",
    "西班牙语",
    "捷克语",
    "意大利语",
    "瑞典语",
    "希腊语",
    "马来语",
    "斯洛伐克语",
    "柬埔寨语",
    "罗马尼亚语",
    "克罗地亚语",
    "乌兹别克语",
    "缅甸语"
]

simple_column_name = "简体中文(源)"
english_column_name = "English"
trans_column_name = "繁體中文"

for target_column in target_columns:
    print(target_column)
    dictionary = {}

    def generate_text(index, data):
        if not pd.isnull(data[target_column]):
            dictionary[str(data[english_column_name])] = data[target_column]
            return index, data[target_column]
        if str(data[english_column_name]) in dictionary.keys():
            return index, dictionary[str(data[english_column_name])]
        if target_column == trans_column_name:
            return index, zhconv.convert(str(data[simple_column_name]), 'zh-tw')
        completion = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {"role": "user", "content": f"Translate the following sentence or word from English to {simple_column_name}: {str(data[english_column_name])}, please directly translate it and do not output any extra content"},
                {"role": "assistant", "content": str(data[simple_column_name])},
                {"role": "user", "content": f"Translate the following sentence or word from English to {trans_column_name}: {str(data[english_column_name])}, please directly translate it and do not output any extra content"},
                {"role": "assistant", "content": zhconv.convert(str(data[simple_column_name]), 'zh-tw')},
                {"role": "user", "content": f"Translate the following sentence or word from English to {target_column}: {str(data[english_column_name])}, please directly translate it and do not output any extra content"}
            ],
            # messages=[
            #     {"role": "user", "content": f"Translate the following sentence from Chinese to {english_column_name}: {str(data[simple_column_name])}, please directly translate it and do not output any extra content"},
            #     {"role": "assistant", "content": str(data[english_column_name])},
            #     {"role": "user", "content": f"Translate the following sentence from Chinese to {trans_column_name}: {str(data[simple_column_name])}, please directly translate it and do not output any extra content"},
            #     {"role": "assistant", "content": str(data[trans_column_name])},
            #     {"role": "user", "content": f"Translate the following sentence from Chinese to {target_column}: {str(data[simple_column_name])}, please directly translate it and do not output any extra content"}
            # ],
            temperature=0
        )
        dictionary[str(data[english_column_name])] = completion.choices[0].message.content
        return index, completion.choices[0].message.content

    from tqdm import tqdm
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import pandas as pd
    to_fix = pd.read_excel(r"/Users/princepride/Downloads/友户通-新增词条-需要翻译-33种-20250722.xlsx")

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

        for future in tqdm(as_completed(futures), total=len(futures)):
            index, output = future.result()

            if output is not None:
                to_fix.at[index, target_column] = output

    # 保存最终结果到 Excel 文件
    to_fix.to_excel(r"/Users/princepride/Downloads/友户通-新增词条-需要翻译-33种-20250722.xlsx", index=False)

繁體中文


  0%|          | 0/14 [00:00<?, ?it/s]/var/folders/zq/y2z7yzsx6hzd7grnc7gwyj340000gn/T/ipykernel_68379/895030355.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '下述選項至少開啟一個:重置密碼是否設置用戶未激活、重置密碼後是否設置強制修改初始密碼' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  to_fix.at[index, target_column] = output
100%|██████████| 14/14 [00:00<00:00, 7234.23it/s]


印尼语


  0%|          | 0/14 [00:00<?, ?it/s]/var/folders/zq/y2z7yzsx6hzd7grnc7gwyj340000gn/T/ipykernel_68379/895030355.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Label SMS (Thai)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  to_fix.at[index, target_column] = output
100%|██████████| 14/14 [00:03<00:00,  4.44it/s]


匈牙利语


...

  0%|          | 0/14 [00:00<?, ?it/s]/var/folders/zq/y2z7yzsx6hzd7grnc7gwyj340000gn/T/ipykernel_68379/895030355.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Predmet e-pošte (kineski)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  to_fix.at[index, target_column] = output
100%|██████████| 14/14 [00:01<00:00,  8.75it/s]


乌兹别克语


  0%|          | 0/14 [00:00<?, ?it/s]/var/folders/zq/y2z7yzsx6hzd7grnc7gwyj340000gn/T/ipykernel_68379/895030355.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Email Mavzusi (Tay)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  to_fix.at[index, target_column] = output
100%|██████████| 14/14 [00:01<00:00,  7.15it/s]


缅甸语


  0%|          | 0/14 [00:00<?, ?it/s]/var/folders/zq/y2z7yzsx6hzd7grnc7gwyj340000gn/T/ipykernel_68379/895030355.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'အီးမေးလ်ခေါင်းစဉ် (ချင်း)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  to_fix.at[index, target_column] = output
100%|██████████| 14/14 [00:03<00:00,  4.50it/s]


# PDF翻译

In [1]:
from dotenv import load_dotenv
load_dotenv()

!pdf2zh "/Users/princepride/Downloads/2025用友大型企业客户成功服务画册（电子版）.pdf" -li zh -lo en -s openai

not in git repo
Namespace(files=['/Users/princepride/Downloads/2025用友大型企业客户成功服务画册（电子版）.pdf'], debug=False, pages=None, vfont='', vchar='', lang_in='zh', lang_out='en', service='openai', output='', thread=4, interactive=False, share=False, flask=False, celery=False, authorized=None, prompt=None, compatible=False, onnx=None, serverport=None, dir=False, config=None, babeldoc=False, skip_subset_fonts=False, ignore_cache=False, mcp=False, sse=False)
[07/31/25 17:52:38] INFO     INFO:pdf2zh.high_level:use font:  ]8;id=75715;file:///opt/miniconda3/envs/ai-trans/lib/python3.12/site-packages/pdf2zh/high_level.py\high_level.py]8;;\:]8;id=830752;file:///opt/miniconda3/envs/ai-trans/lib/python3.12/site-packages/pdf2zh/high_level.py#423\423]8;;\
                             /Users/princepride/.cache/babeldo                  
                             c/fonts/GoNotoKurrent-Regular.ttf                  
100%|███████████████████████████████████████████| 41/41 [00:51<00:00,  1.25s/it]
MuPDF 

In [2]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/", api_key="")
target_column = "印尼语"

def generate_text(index, data):
    if not pd.isnull(data[target_column]):
        return index, data[target_column]
    completion = client.chat.completions.create(
        model='gemini-1.5-flash',
        messages=[
            {"role": "user", "content": f"Translate English to Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
            {"role": "assistant", "content": str(data['简体中文(源)'])},
            {"role": "user", "content": f"Translate English to Traditional Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
            {"role": "assistant", "content": str(data['繁体中文（译）'])},
            {"role": "user", "content": f"Translate English to {target_column}: {str(data['English(译)'])}, please directly translate it and do not output any extra content"}
        ],
        temperature=0
    )
    return index, completion.choices[0].message.content

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
to_fix = pd.read_excel(r"D:\Projects\ai-translator\src\友互通\友互通-全.xlsx")

with ThreadPoolExecutor(max_workers=1000) as executor:
    futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

    for future in tqdm(as_completed(futures), total=len(futures)):
        index, output = future.result()

        if output is not None:
            to_fix.at[index, target_column] = output

# 保存最终结果到 Excel 文件
to_fix.to_excel(r"D:\Projects\ai-translator\src\友互通\友互通-全.xlsx", index=False)

(0, 'matahari\n')

# 术语表合并

In [3]:
import pandas as pd

original_path = r"D:\Projects\ai-translator\src\models\API\chatgpt-4o-mini\glossary.xlsx"
new_path = r"C:\Users\wangz\Downloads\用友技术类常用术语表 for MT-20250320.xlsx"
output_path = "glossary.xlsx"

def merge_translation_tables(original_path, new_path, output_path, original_language, target_language):
    # 读取原始术语表和新增术语表
    original_df = pd.read_excel(original_path)
    new_df = pd.read_excel(new_path)
    
    # 确保列名存在
    if original_language not in original_df.columns or target_language not in original_df.columns:
        raise ValueError("Original language and target language columns must exist in the original table")
    if original_language not in new_df.columns or target_language not in new_df.columns:
        raise ValueError("Original language and target language columns must exist in the new table")
    
    # 过滤掉原始术语为空的行
    original_df = original_df.dropna(subset=[original_language])
    new_df = new_df.dropna(subset=[original_language])
    
    # 找出新增的术语（原术语表中不存在的）
    existing_terms = set(original_df[original_language].astype(str))
    filtered_new_df = new_df[~new_df[original_language].astype(str).isin(existing_terms)]
    
    # 合并数据表
    merged_df = pd.concat([original_df, filtered_new_df], ignore_index=True)
    
    # 保存合并后的数据
    merged_df.to_excel(output_path, index=False)
    
    print(f"术语表已合并，保存至: {output_path}")
    return merged_df

merge_translation_tables(original_path, new_path, output_path, "Chinese", "English")

术语表已合并，保存至: new_glossary.xlsx


,Chinese,English,Thai
0,会计服务,Accounting,บริการด้านบัญชี
1,企业绩效,Enterprise Performance Management,ผลการดำเนินธุรกิจ
2,业财综合分析,Financial Analytics,การวิเคราะห์การเงินแบบบูรณาการ
3,数智合同,Contract Lifecycle Management,สัญญาอัจฉริยะดิจิทัล
4,采购供应,Material Mgmt,การจัดหาซัพพลาย
...,...,...,...
5048,缓存机制,Caching Mechanism,NaN
5049,管理类角色,Administrative Role,NaN
5050,业务类角色,Business Role,NaN
5051,应用发布,Application Release,NaN


# 文件夹批翻译

In [3]:
from typing import Optional
import time
from pptx import Presentation
from main_ui import translate, word_to_markdown, extract_complex_blocks, restore_complex_blocks, markdown_to_word
import openpyxl
import zipfile
import shutil
import os

def translate_folder_with_structure(
    source_folder_path: str,
    output_parent_dir: str,
    selected_model: Optional[str],
    selected_lora_model: Optional[str],
    selected_gpu: Optional[str],
    batch_size: int,
    original_language: Optional[str],
    target_language: Optional[str]
):
    """
    批量翻译指定文件夹内的所有文件，并保持原始目录结构。

    :param source_folder_path: 要翻译的源文件夹路径。
    :param output_parent_dir: 用于存放生成结果的父目录。
    :param selected_model: 翻译模型。
    :param selected_lora_model: LoRA模型。
    :param selected_gpu: 使用的GPU。
    :param batch_size: 批处理大小。
    :param original_language: 源语言。
    :param target_language: 目标语言。
    :return: 一个包含处理信息和最终ZIP文件路径的元组 (message, zip_path)。
    """
    start_time = time.time()

    if not os.path.isdir(source_folder_path):
        return "源路径不是一个有效的文件夹", None

    # 1. 设置输出路径
    base_folder_name = os.path.basename(os.path.normpath(source_folder_path))
    # 创建一个带时间戳的、包含所有翻译后文件的根目录，以避免冲突
    output_root_folder = os.path.join(output_parent_dir, f"{base_folder_name}_translated_{int(start_time)}")
    
    # 用于存放Word转Markdown时抽取的图片
    temp_image_dir = os.path.join(output_root_folder, "temp_images_from_docx")
    os.makedirs(temp_image_dir, exist_ok=True)
    
    print(f"源文件夹: {source_folder_path}")
    print(f"输出根目录: {output_root_folder}")

    processed_files_paths = []
    
    # 2. 递归遍历源文件夹
    for root, _, files in os.walk(source_folder_path):
        for filename in files:
            source_file_path = os.path.join(root, filename)
            
            # 计算并创建在输出目录中对应的路径结构
            relative_path = os.path.relpath(source_file_path, source_folder_path)
            output_file_path_dest = os.path.join(output_root_folder, relative_path)
            
            # 确保目标目录存在
            os.makedirs(os.path.dirname(output_file_path_dest), exist_ok=True)

            file_name_only, file_ext = os.path.splitext(filename)

            try:
                # 3. 根据文件类型选择相应的处理逻辑 (移植自您的代码)
                if file_ext.lower() == '.pptx':
                    # --- PPTX 处理逻辑 ---
                    prs = Presentation(source_file_path)
                    run_list = []
                    text_list = []
                    
                    # 您的 extract_text_from_shape 函数非常棒，直接使用
                    def extract_text_from_shape(shape, run_list, text_list):
                        if hasattr(shape, "text_frame") and shape.text_frame is not None:
                            for paragraph in shape.text_frame.paragraphs:
                                for run in paragraph.runs:
                                    run_list.append(run); text_list.append(run.text)
                        elif getattr(shape, "has_table", False):
                            for row in shape.table.rows:
                                for cell in row.cells:
                                    if cell.text_frame is not None:
                                        for paragraph in cell.text_frame.paragraphs:
                                            for run in paragraph.runs:
                                                run_list.append(run); text_list.append(run.text)
                        elif hasattr(shape, "shapes"):
                            for sub_shape in shape.shapes:
                                extract_text_from_shape(sub_shape, run_list, text_list)
                                                
                    for slide in prs.slides:
                        for shape in slide.shapes:
                            extract_text_from_shape(shape, run_list, text_list)

                    if text_list:
                        translated_segments = translate(text_list, selected_model, selected_lora_model, selected_gpu, batch_size, original_language, target_language)
                        for run, translated in zip(run_list, translated_segments):
                            run.text = translated[0]["generated_translation"]
                    
                    prs.save(output_file_path_dest)
                    processed_files_paths.append(output_file_path_dest)
                    print(f"已翻译: {source_file_path} -> {output_file_path_dest}")

                elif file_ext.lower() in ['.xlsx', '.xls']:
                    # --- Excel 处理逻辑 ---
                    workbook = openpyxl.load_workbook(source_file_path)
                    texts_to_translate, cell_locations = [], []

                    for sheet_name in workbook.sheetnames:
                        sheet = workbook[sheet_name]
                        for row in sheet.iter_rows():
                            for cell in row:
                                if cell.value:
                                    texts_to_translate.append(str(cell.value))
                                    cell_locations.append((sheet_name, cell.row, cell.column))
                    
                    if texts_to_translate:
                        translated_results = translate(texts_to_translate, selected_model, selected_lora_model, selected_gpu, batch_size, original_language, target_language)
                        for i, location in enumerate(cell_locations):
                            translated_text = translated_results[i][0]['generated_translation']
                            sheet_name, row, col = location
                            workbook[sheet_name].cell(row=row, column=col, value=translated_text)
                    
                    workbook.save(output_file_path_dest)
                    processed_files_paths.append(output_file_path_dest)
                    print(f"已翻译: {source_file_path} -> {output_file_path_dest}")

                elif file_ext.lower() in ['.docx', '.md']:
                    # --- Docx/Markdown 处理逻辑 ---
                    md_content = ""
                    if file_ext.lower() == '.docx':
                        md_content = word_to_markdown(source_file_path, output_dir=temp_image_dir)
                    else: # .md
                        with open(source_file_path, 'r', encoding='utf-8') as f:
                            md_content = f.read()

                    clean_md, protected_blocks = extract_complex_blocks(md_content)
                    text_to_translate = [p for p in clean_md.split('\n\n') if p.strip()]

                    if text_to_translate:
                        translated_results = translate(text_to_translate, selected_model, selected_lora_model, selected_gpu, batch_size, original_language, target_language)
                        translation_map = {original: result[0]['generated_translation'] for original, result in zip(text_to_translate, translated_results)}
                        
                        temp_translated_content = [translation_map.get(para, para) for para in clean_md.split('\n\n')]
                        translated_content = '\n\n'.join(temp_translated_content)
                    else:
                        translated_content = clean_md
                    
                    final_md_content = restore_complex_blocks(translated_content, protected_blocks)
                    
                    if file_ext.lower() == '.docx':
                        markdown_to_word(final_md_content, output_file_path_dest, image_base_dir=temp_image_dir)
                    else: # .md
                        with open(output_file_path_dest, 'w', encoding='utf-8') as f:
                            f.write(final_md_content)
                    
                    processed_files_paths.append(output_file_path_dest)
                    print(f"已翻译: {source_file_path} -> {output_file_path_dest}")

                else:
                    # 4. 对于不支持翻译的文件类型，直接复制到目标位置
                    shutil.copy2(source_file_path, output_file_path_dest)
                    print(f"已复制: {source_file_path} -> {output_file_path_dest}")

            except Exception as e:
                print(f"处理文件 {source_file_path} 时发生严重错误: {e}")
                import traceback
                traceback.print_exc()
                continue # 跳过此文件，继续处理下一个


    end_time = time.time()
    duration = int(end_time - start_time)
    
    return f"处理完成，总耗时: {duration}秒。{len(processed_files_paths)}个文件被翻译。"

In [4]:
translate_folder_with_structure(
    source_folder_path=r'/Users/princepride/Downloads/Canary Documentation',
    output_parent_dir=r'/Users/princepride/Downloads',
    selected_model='gpt-4o-mini',
    selected_lora_model=None,
    selected_gpu='0',
    batch_size=1,
    original_language='English',
    target_language=['Chinese']
)

源文件夹: /Users/princepride/Downloads/Canary Documentation
输出根目录: /Users/princepride/Downloads/Canary Documentation_translated_1753775533
已复制: /Users/princepride/Downloads/Canary Documentation/.DS_Store -> /Users/princepride/Downloads/Canary Documentation_translated_1753775533/.DS_Store
已翻译: /Users/princepride/Downloads/Canary Documentation/CommodityFourServer/Program.md -> /Users/princepride/Downloads/Canary Documentation_translated_1753775533/CommodityFourServer/Program.md
已翻译: /Users/princepride/Downloads/Canary Documentation/CommodityFourServer/ServerForm.Designer.md -> /Users/princepride/Downloads/Canary Documentation_translated_1753775533/CommodityFourServer/ServerForm.Designer.md
已翻译: /Users/princepride/Downloads/Canary Documentation/CommodityFourServer/ServerForm.md -> /Users/princepride/Downloads/Canary Documentation_translated_1753775533/CommodityFourServer/ServerForm.md
已翻译: /Users/princepride/Downloads/Canary Documentation/CommodityFourServer/Properties/Resources.Designer.md -

'处理完成，总耗时: 10192秒。1434个文件被翻译。'